Experimenting with Bayesian optimisation to produce adversarial examples.


References:
- General guide to bayesian optimization:
    https://distill.pub/2020/bayesian-optimization/
- Paper on using bayesian optimization in hard label settings:
https://arxiv.org/pdf/2007.07210.pdf
- Another paper with a different objective function that could be used:
https://arxiv.org/abs/1807.04457

Things to still do:
- Incorporate model wrapper to ensure samples are valid
- Look at rounding booleans, etc to ensure samples are valid
- Further adjust distance function to take into account categorical features
- Potentially adjust weighting vector to penalise changes to some fields more than others?
- Adjust objective function to use both failed_prob and distance

Scikit-optimize: library for optimization, including functions for Bayesian optimization.

In [1]:
import skopt
from skopt.plots import plot_convergence
import os
import pickle
import pandas as pd
import numpy as np
import math
from processing.ModelWrapper import ModelWrapper
from processing.PostcodeEncoder import PostcodeEncoder


C:\Users\ral20\anaconda3\envs\branch-env\lib\site-packages\IPython\core\interactiveshell.py:3361: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Step 1: defining search space.
Initially do so by taking min/max for each field in test data.

In [2]:
#Load in test data
data_dir = os.path.dirname(os.getcwd())
with open(os.path.join(data_dir, "Processing", r"train_preproc.p"), 'rb') as data_file:
    train_data = pickle.load(data_file)
X_train, y_train = train_data
# forgot in preprocessing: convert from bool to int
X_train['hasGNotice'] = X_train['hasGNotice'].apply(int)

In [3]:
pd.set_option("display.max_rows", 345)
X_train.head(10).transpose()

,2472532,1236111,1186252,2841959,1856553,910107,3048539,2623910,1485615,1806284
AccountsAccountCategory,9.000000e+00,1.300000e+01,1.300000e+01,1.000000e+01,9.000000e+00,1.300000e+01,1.000000e+01,3.000000e+00,9.000000e+00,1.300000e+01
AccountsAccountRefDay,3.100000e+01,3.000000e+01,3.100000e+01,3.000000e+01,3.100000e+01,3.100000e+01,3.100000e+01,3.100000e+01,3.100000e+01,3.100000e+01
AccountsAccountRefMonth,3.000000e+00,4.000000e+00,1.200000e+01,4.000000e+00,5.000000e+00,3.000000e+00,8.000000e+00,1.000000e+01,1.200000e+01,3.000000e+00
CompanyCategory,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
CompanyNameCountNum,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
CompanyNameCountX,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
CompanyNameLen,1.200000e+01,1.500000e+01,2.000000e+01,2.300000e+01,1.800000e+01,1.800000e+01,1.900000e+01,1.500000e+01,2.300000e+01,3.200000e+01
CompanyNameWordLen,2.000000e+00,2.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00
Field1014,6.388402e-16,2.746206e-01,1.110084e+00,6.388402e-16,1.763889e-01,5.934338e-01,6.388402e-16,-2.064843e+00,6.621412e-02,3.535633e-01
Field1129,-8.717939e-16,-8.717939e-16,-8.717939e-16,-8.717939e-16,-4.761692e-01,9.043614e-01,-8.717939e-16,-8.717939e-16,-1.128138e+00,-8.717939e-16


In [4]:
min_max = pd.DataFrame(index=["min", "max"], columns=X_train.columns)

for column in X_train.columns:
    min_max[column]["max"] = X_train[column].max(axis=0)
    min_max[column]["min"] = X_train[column].min(axis=0)


pd.set_option("display.max_rows", 345)

min_max

,AccountsAccountCategory,AccountsAccountRefDay,AccountsAccountRefMonth,CompanyCategory,CompanyNameCountNum,CompanyNameCountX,CompanyNameLen,CompanyNameWordLen,Field1014,Field1129,...,oac1,oac11,oac2,oseast1m,osnrth1m,ru11ind,dAccountsTimeGap,dConfStmtTimeGap,dReturnsTimeGap,OtherCompInPcd
min,0,1.0,1.0,0,0,0,5,1,-2.255231,-2.194843,...,1.0,0,0,-8.678779,-6.298319,0,-0.167123,0.038356,0.352459,-0.847661
max,15,31.0,12.0,1,32,5,103,16,4.06933,3.014633,...,8.0,75,25,1.731645,3.226095,26,2.728535,1.69589,2.616438,3.531579


In [5]:
#load in model to attack - try using rf instead, modules not importing properly
model = pickle.load(open(os.path.join(data_dir, "fitted_models", "rf_0.929"), "rb"))

C:\Users\ral20\anaconda3\envs\branch-env\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\ral20\anaconda3\envs\branch-env\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [6]:
# Need to get search space from min_max array
search_space = list(map(tuple, min_max.transpose().to_records(index=False)))

In [15]:
derived_features = ["oseast1m", "osnrth1m", "cty", "lat", "long", "ru11ind", "oac11",
                    "country", "oac1", "oac2", "imdu", "OtherCompInPcd"]

incomplete_columns = X_train.columns.drop(derived_features)
incomplete_columns = incomplete_columns.insert(incomplete_columns.size, 'pcd')
print(incomplete_columns)

#Adjusting the distance function to take into account whether or not data is categorical (ie whether the
# distance means anything, or if what matters is simply if there is a change.)
cat_features = ["AccountsAccountCategory", "CompanyCategory", "RegAddressCountry", "pcd"]
cat_features_index = [incomplete_columns.get_loc(col_name) for col_name in cat_features]
print(cat_features_index)

Index(['AccountsAccountCategory', 'AccountsAccountRefDay',
       'AccountsAccountRefMonth', 'CompanyCategory', 'CompanyNameCountNum',
       'CompanyNameCountX', 'CompanyNameLen', 'CompanyNameWordLen',
       'Field1014', 'Field1129',
       ...
       'hasF69', 'hasF70', 'hasGNotice', 'nSIC', 'namechanged', 'namechanged2',
       'dAccountsTimeGap', 'dConfStmtTimeGap', 'dReturnsTimeGap', 'pcd'],
      dtype='object', length=103)
[0, 3, 48, 102]


In [8]:
#Rouding for integers/booleans
X_train.dtypes
X_train.dtypes["oac2"] == np.dtype('int64')

integer_features = []

for i in range(0,X_train.dtypes.size):
    if(X_train.dtypes[i] == np.dtype('int64')):
        integer_features.append(X_train.dtypes.index[i])
        
integer_features

boolean_features = []

for i in range(0,X_train.dtypes.size):
    if (X_train.dtypes[i] == np.dtype('int32')) or (X_train.dtypes[i] == np.dtype('bool')) :
        boolean_features.append(X_train.dtypes.index[i])
        
boolean_features.remove('eAccountsAccountCategory')
boolean_features.remove('eCompanyCategory')
boolean_features

integer_features.append('eAccountsAccountCategory')
integer_features.append('eCompanyCategory')
integer_features

['CompanyNameCountNum',
 'CompanyNameCountX',
 'CompanyNameLen',
 'CompanyNameWordLen',
 'MortgagesNumMortCharges',
 'MortgagesNumMortOutstanding',
 'MortgagesNumMortPartSatisfied',
 'MortgagesNumMortSatisfied',
 'hasGNotice',
 'eAccountsAccountCategory',
 'eCompanyCategory']

An alternative way of building the search space - looking at the standard deviation for each field to limit the search space and ensure delta is smaller.

Doesn't look at standard deviation for categorical features which are not hierarchical.

In [9]:
# An alternative search_space - looking at standard deviation for each field
sd = pd.DataFrame(index=["sd"], columns=X_train.columns)

for col_name in X_train.columns:
    sd[col_name] = X_train[col_name].std(axis=0)

# Finding min/max for categorical features:
min_max_cat = pd.DataFrame(index=["min", "max"], columns=cat_features)
for col_name in cat_features:
    if col_name == 'pcd':
        min_max_cat[col_name]["max"] = PostcodeEncoder().num_postcodes()
        min_max_cat[col_name]["min"] = 0
    else:
        min_max_cat[col_name]["max"] = X_train[col_name].max(axis=0)
        min_max_cat[col_name]["min"] = X_train[col_name].min(axis=0)
    
print(min_max_cat)
    

def construct_search_space(original):
    search_space = list()
    for col_name in original.columns:
        if col_name in cat_features:
            search_space.append(tuple(min_max_cat[col_name]))
        elif col_name in boolean_features:
            search_space.append((0,1))
        else:
            original_value = original[sd.columns.get_loc(col_name)]
            col_sd = sd.at["sd", col_name]
            if col_name in integer_features:
                lower = math.floor(original_value-0.1*col_sd)
                upper = math.ceil(original_value+0.1*col_sd)
            else:
                lower = original_value-0.1*col_sd
                upper = original_value+0.1*col_sd
            search_space.append((lower, upper))
        #print(search_space)
    return search_space

    AccountsAccountCategory CompanyCategory RegAddressCountry      pcd
min                       0               0                 0        0
max                      15               1                 6  2632804


In [10]:

print(incomplete_columns)

Index(['AccountsAccountCategory', 'AccountsAccountRefDay',
       'AccountsAccountRefMonth', 'CompanyCategory', 'CompanyNameCountNum',
       'CompanyNameCountX', 'CompanyNameLen', 'CompanyNameWordLen',
       'Field1014', 'Field1129',
       ...
       'hasF69', 'hasF70', 'hasGNotice', 'nSIC', 'namechanged', 'namechanged2',
       'dAccountsTimeGap', 'dConfStmtTimeGap', 'dReturnsTimeGap', 'pcd'],
      dtype='object', length=103)


In [11]:
#something new - only attacking the postcode and deriving the other features
# change to search space - uses postcode, no derived fields
# treat postcode as categorical rather than numerical - we don't care what it changes to

# takes original data point with derived fields removed
def construct_derived_search_space(original):
    search_space = list()
    print(original.index)
    for col_name in original.index:
        if col_name in cat_features:
            search_space.append(tuple(min_max_cat[col_name]))
        elif col_name in boolean_features:
            search_space.append((0,1))
        else:
            original_value = original[col_name]
            col_sd = sd.at["sd", col_name]
            if col_name in integer_features:
                lower = math.floor(original_value-0.1*col_sd)
                upper = math.ceil(original_value+0.1*col_sd)
            else:
                lower = original_value-0.1*col_sd
                upper = original_value+0.1*col_sd
            search_space.append((lower, upper))
        #print(search_space)
    return search_space


In [26]:
# weight some features differently in distance function
# weight of 1 for categorical features
#no_postcode = incomplete_columns.delete('pcd')
#print(no_postcode)
weighting_vector = np.zeros(incomplete_columns.size)
for i in range(0, weighting_vector.size):
    if i in cat_features_index:
        weighting_vector[i] = 1/X_train.iloc[i].mean()+0.01
    else:
        weighting_vector[i] = 1
#weighting_vector = np.array([1/(col.mean()+0.01) for col in [X_train[col_name] for col_name in incomplete_columns.drop('pcd')]])
#for index in cat_features_index:
#    weighting_vector[index] = 1
    
weighting_vector

array([0.01557611, 1.        , 1.        , 0.01553499, 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.01552157, 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.     

In [27]:
def build_df(X):
    if(len(X) == len(X_train.columns)):
       return pd.DataFrame(data=[X], columns=X_train.columns)
    elif(len(X) == len(incomplete_columns)):
       return pd.DataFrame(data=[X], columns=incomplete_columns)
    # else helpful error message

def get_arr(X):
    return X.values[0]

In [28]:
# TODO: standardise all features so that this is just absolute distance
# Adjusted to take into account categorical features
def distance_function(X, company, weighting_vector):
    d = abs(X-company)
    for index in cat_features_index:
        if(X[index] != company[index]):
            d[index] = 1
        else:
            d[index] = 0
    return (d @ weighting_vector)

In [29]:
""" ideas:
1) constrain search space to [ex - delta, ex + delta] either manually or by specifying the search space
Have the objective function just return the failed probability, and optimise it until it is small
2) add reasonable distance function to objective


Model wrapper:
 - find postcode fields from postcode
 - round boolean fields

"""

# Note to self - have messed with types here
# other note - needs changing, often first sample with 0 delta will be minimum found
# getting there tho
def compute_objective(X, true_company, wrapper):
    print(1)
    X_df = build_df(X)
    delta = distance_function(X, true_company, weighting_vector)
    # completion of data is done implicitly by the wrapper here
    failed_prob = wrapper.predict_proba(X_df)[0,1]
#    print("delta: ", delta)
    # experimenting with the number here - trying 15, as delta almost always seems to be less than that
    # other ideas - ignore failed_prob once <0.5 - minimising delta is more important than minimising failed_prob
    # once we have successfully tricked the model
    obj = 10*failed_prob * (delta + 15)
#    print("objective: ", obj)
    if wrapper.predict(X_df)[0] == 0:
#        print("model successfully fooled into thinking company is safe")
#        print("probability of failing: %s" % failed_prob)
        # attempting to always return obj instead of -1 - decrease failed_prob and delta both?
        return obj

#    print("model not tricked")
#    print("probability of failing: %s" % failed_prob)
#    print()
    return obj

A second attempt at an objective function - focus on minimising delta rather than failed_prob.

In [16]:
# alternative - not considering failed prob, only model output and delta
def compute_objective1(X, true_company, wrapper):
    X_df = build_df(X)
    delta = distance_function(X, true_company, weighting_vector)
    failed_prob = wrapper.predict_proba(X_df)[0,1]
#    print("delta: ", delta)
    # experimenting with the number here - trying 150, as delta almost always seems to be less than that
    if wrapper.predict(X_df)[0] == 0:
#        print("model successfully fooled into thinking company is safe")
#        print("probability of failing: %s" % failed_prob)
        return delta

#    print("model not tricked")
#    print("probability of failing: %s" % failed_prob)
#    print()
    # 150 seems a reasonable number, so it's almost always better to have a sample that successfully tricks the model
    # even if it's quite far from the starting sample
    return 1500

In [17]:
# objective function incorporating both failed_prob and delta - smoother function
# numbers will need adjusting to find something reasonable
def compute_objective2(X, true_company, wrapper):
    X_df = build_df(X)
    delta = distance_function(X, true_company, weighting_vector)
    failed_prob = wrapper.predict_proba(X_df)[0,1]
#    print("delta: ", delta)
    if wrapper.predict(X_df)[0] == 0:
#        print("model successfully fooled into thinking company is safe")
        # Focus on minimising delta
        obj = 100*failed_prob + delta
    else:
#        print("model not tricked")
        # Focus on minimising prob
        obj = 1000 * failed_prob + delta
#    print("probability of failing: %s" % failed_prob)
#    print("objective value: %s" % obj)
#    print()
    return obj

In [30]:
encoder = PostcodeEncoder()

# objective function incorporating both failed_prob and delta - smoother function
# making numbers smaller
def compute_objective3(X, true_company, wrapper):
    X_df = build_df(X)
    X_df['pcd'] = encoder.search_decode(X_df['pcd'])
    delta = distance_function(X, true_company, weighting_vector)
    failed_prob = wrapper.predict_proba(X_df)[0,1]
#    print("delta: ", delta)
    if wrapper.predict(X_df)[0] == 0:
#        print("model successfully fooled into thinking company is safe")
        # Focus on minimising delta
        obj = 10*failed_prob + delta
    else:
#        print("model not tricked")
        # Focus on minimising prob
        obj = 100 * failed_prob + delta
#    print("probability of failing: %s" % failed_prob)
#    print("objective value: %s" % obj)
#    print()
    return obj

In [31]:
class FinishedCallBack(skopt.callbacks.EarlyStopper):
    def __call__(self, result):
      """
      Messy code
      Parameters
      ----------
      result : `OptimizeResult`, scipy object
          The optimization as a OptimizeResult object.
      """
      return self._criterion(result)
    def _criterion(self, result):
        return (result.fun == -1)

In [32]:
failed_companies = X_train.loc[y_train == 1]
failed_companies.shape
for i in range(0, 957):
    sample = failed_companies.iloc[i].values.reshape(1,-1)
    print(model.predict)

<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_st

<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_st

<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_st

<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)>
<bound method ForestClassifier.predict of RandomForestClassifier(max_depth=7, n_estimators=20, random_st

In [33]:

print(failed_companies.shape[1])
true_positives = []

for i in range(0, failed_companies.shape[0]):
    sample = failed_companies.iloc[i].values.reshape(1,-1)
    if model.predict(sample) == 1:
        true_positives.append(sample)
        
len(true_positives)
true_positives

114


[array([[ 9.00000000e+00,  3.10000000e+01,  1.20000000e+01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          3.00000000e+01,  3.00000000e+00,  6.38840193e-16,
         -8.71793851e-16, -3.10052334e-16,  1.77652248e-16,
         -1.56558199e-16,  2.88356997e-16,  1.84551297e+00,
          1.44482253e+00, -4.41993037e-16,  0.00000000e+00,
          1.10871410e+00,  2.38743081e-16,  1.71058197e-16,
          1.27815240e+00,  1.12659298e+00,  1.14331317e+00,
          1.78660176e+00, -3.03130231e-16,  1.71546708e+00,
          1.84904654e+00,  1.81159904e+00, -5.95638058e-16,
          1.10900000e+03, -2.86734765e-16,  6.41083183e-16,
          3.02044503e-16,  2.40922250e-17,  1.33467762e+00,
         -2.80954398e-16,  4.55325202e-16,  1.06496582e+00,
          1.02015892e+00,  1.26477639e+00,  1.10678915e+00,
          0.00000000e+00, -1.42540127e+01,  5.10000000e+01,
          2.00000000e+00,  0.00000000e+00,  4.90000000e+01,
          2.00000000e+00,  9.80000000e+0

In [34]:
# get initial point (company data we want to adjust)
# for now, just use the first company in the test data that is predicted to fail
fail = False
i = 0
while(not(fail)):
    if(y_train[i] == 1):
        if(model.predict(X_train.iloc[i].values.reshape(1, -1)) == 1):
            example = X_train.iloc[i].values.reshape(1,-1)
            fail = True
    if (not(fail)): i += 1

print(model.predict(example))
print(y_train[i])
example = X_train.iloc[i]
example = example.drop(labels = derived_features, axis=0)
print(example)
#example = example.values.reshape(1,-1).flatten()
print(len(example))
example
#objective(example)
# need to add postcode as well here, and convert to search_encoding

[1]
1
AccountsAccountCategory          9.000000e+00
AccountsAccountRefDay            3.100000e+01
AccountsAccountRefMonth          1.200000e+01
CompanyCategory                  0.000000e+00
CompanyNameCountNum              0.000000e+00
CompanyNameCountX                0.000000e+00
CompanyNameLen                   3.000000e+01
CompanyNameWordLen               3.000000e+00
Field1014                        6.388402e-16
Field1129                       -8.717939e-16
Field1522                       -3.100523e-16
Field1631                        1.776522e-16
Field17                         -1.565582e-16
Field1865                        2.883570e-16
Field1871                        1.845513e+00
Field1885                        1.444823e+00
Field1977                       -4.419930e-16
Field2267                        0.000000e+00
Field2298                        1.108714e+00
Field2304                        2.387431e-16
Field2316                        1.710582e-16
Field2447                   

AccountsAccountCategory          9.000000e+00
AccountsAccountRefDay            3.100000e+01
AccountsAccountRefMonth          1.200000e+01
CompanyCategory                  0.000000e+00
CompanyNameCountNum              0.000000e+00
CompanyNameCountX                0.000000e+00
CompanyNameLen                   3.000000e+01
CompanyNameWordLen               3.000000e+00
Field1014                        6.388402e-16
Field1129                       -8.717939e-16
Field1522                       -3.100523e-16
Field1631                        1.776522e-16
Field17                         -1.565582e-16
Field1865                        2.883570e-16
Field1871                        1.845513e+00
Field1885                        1.444823e+00
Field1977                       -4.419930e-16
Field2267                        0.000000e+00
Field2298                        1.108714e+00
Field2304                        2.387431e-16
Field2316                        1.710582e-16
Field2447                        1

##### Actual Bayesian optimization:

In [35]:
starting_example = example
print(starting_example)
x0 = list(starting_example)
print(x0)
wrapped = ModelWrapper(model)
def objective(X):
    return compute_objective2(X, starting_example, wrapped)

search_space1 = construct_derived_search_space(starting_example)

# sometimes gets error here - keep rerunning and it will work after a while.
# Adjusted to not use a callback - need to keep going even if we find a sample that works, in order to minimise
# distance from original
#cb = FinishedCallBack()
res = skopt.gp_minimize(objective, search_space1, x0 = x0, y0 = objective(x0))
plot_convergence(res)
minimum = res.x


AccountsAccountCategory          9.000000e+00
AccountsAccountRefDay            3.100000e+01
AccountsAccountRefMonth          1.200000e+01
CompanyCategory                  0.000000e+00
CompanyNameCountNum              0.000000e+00
CompanyNameCountX                0.000000e+00
CompanyNameLen                   3.000000e+01
CompanyNameWordLen               3.000000e+00
Field1014                        6.388402e-16
Field1129                       -8.717939e-16
Field1522                       -3.100523e-16
Field1631                        1.776522e-16
Field17                         -1.565582e-16
Field1865                        2.883570e-16
Field1871                        1.845513e+00
Field1885                        1.444823e+00
Field1977                       -4.419930e-16
Field2267                        0.000000e+00
Field2298                        1.108714e+00
Field2304                        2.387431e-16
Field2316                        1.710582e-16
Field2447                        1

ValueError: list.remove(x): x not in list

In [ ]:
# Check the ditance to the original example
distance_function(minimum, starting_example, weighting_vector)

In [ ]:
minimum

In [ ]:
objective(minimum)

Ideas: 
- multiply failed prob by 10 instead of 100 (then adjust the failed case as well)
- test on all 4 true negatives in the training set and compare the different functions
- also test on false positives in the training set - look at minimising the failed_prob to make them 'safer'
- try with xgb if I can install it?
- compare adversarial sample with original sample - what does a delta of 50 actually mean?
- check boolean fields (should be able to just limit search space, right?)
- try limiting the number of calls for the optimisation to see if we can increase speed without impacting performance too much
    

In [ ]:
#Testing different objective functions for every true negative we have
for starting_example in true_positives:
    starting_example = starting_example.flatten()
    def objective(X):
        return compute_objective(X, true_company=starting_example)

    search_space1 = construct_search_space(starting_example)
    x0 = list(starting_example)
    res = skopt.gp_minimize(objective, search_space1, x0 = x0, y0 = objective(x0), n_calls=30)
    plot_convergence(res)
    minimum = res.x
    print("Objective function: %s" % objective(minimum))
    X_df = build_df(minimum)
    delta = distance_function(minimum, starting_example, weighting_vector)
    failed_prob = model.predict_proba(X_df)[0,1]
    print("Delta: %s" % delta)
    print("Failed prob: %s" % failed_prob)
    print()

In [ ]:
#Testing different objective functions for every true negative we have
for starting_example in true_positives:
    starting_example = starting_example.flatten()
    def objective(X):
        return compute_objective1(X, true_company=starting_example)

    search_space1 = construct_search_space(starting_example)
    x0 = list(starting_example)
    res = skopt.gp_minimize(objective, search_space1, x0 = x0, y0 = objective(x0), n_calls=50)
    plot_convergence(res)
    minimum = res.x
    print("Objective function: %s" % objective(minimum))
    X_df = build_df(minimum)
    delta = distance_function(minimum, starting_example, weighting_vector)
    failed_prob = model.predict_proba(X_df)[0,1]
    print("Delta: %s" % delta)
    print("Failed prob: %s" % failed_prob)
    print()

In [ ]:
#Testing different objective functions for every true negative we have
for starting_example in true_positives:
    starting_example = starting_example.flatten()
    def objective(X):
        return compute_objective2(X, true_company=starting_example)

    search_space1 = construct_search_space(starting_example)
    x0 = list(starting_example)
    res = skopt.gp_minimize(objective, search_space1, x0 = x0, y0 = objective(x0), n_calls=50)
    plot_convergence(res)
    minimum = res.x
    print("Objective function: %s" % objective(minimum))
    X_df = build_df(minimum)
    delta = distance_function(minimum, starting_example, weighting_vector)
    failed_prob = model.predict_proba(X_df)[0,1]
    print("Delta: %s" % delta)
    print("Failed prob: %s" % failed_prob)
    print()

In [ ]:
#Testing different objective functions for every true negative we have
for starting_example in true_positives:
    starting_example = starting_example.flatten()
    def objective(X):
        return compute_objective3(X, true_company=starting_example)

    search_space1 = construct_search_space(starting_example)
    x0 = list(starting_example)
    res = skopt.gp_minimize(objective, search_space1, x0 = x0, y0 = objective(x0), n_calls=50)
    plot_convergence(res)
    minimum = res.x
    print("Objective function: %s" % objective(minimum))
    X_df = build_df(minimum)
    delta = distance_function(minimum, starting_example, weighting_vector)
    failed_prob = model.predict_proba(X_df)[0,1]
    print("Delta: %s" % delta)
    print("Failed prob: %s" % failed_prob)
    print()

In [ ]:
print("Original          Adversarial")
for i in range(0,115):
    print("%s       %s" % (true_positives[3][0][i], minimum[i]))

In [ ]:
true_positives[3][0][0]

Thinking about postcode:
 - actual data is proper encoded
 - need to operate on straightforward encoding from csv file
 - search space: max/min from csv
 - convert to straightforward -> convert back to string for data fill in -> convert to encoded and do prediction

In [6]:
X_train['Field2823'].head(50)

NameError: name 'X_train' is not defined